In [1]:
import numpy as np
import pandas as pd

from statistics import mean

from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import (
    cross_validate, train_test_split, GridSearchCV, learning_curve, validation_curve
)
from sklearn.metrics import classification_report, accuracy_score

# Import dataset

In [2]:
compressed_final = pd.read_csv("../DataFormating/compressed_final.csv")
final = pd.read_csv("../DataFormating/final.csv")

# Setup `X`, `y` data for training / testing

In [4]:
compressed_final.columns[:10]

Index(['Stage', 'Home Team Name', 'Home Team Goals', 'Away Team Goals',
       'Away Team Name', 'Attendance', 'Half-time Home Goals',
       'Half-time Away Goals', 'Home Team Initials', 'Away Team Initials'],
      dtype='object')

In [5]:
X = compressed_final.drop(["Home Team Goals", "Away Team Goals", 
                           "Half-time Home Goals", "Half-time Away Goals", 
                           "Home Team Initials", "Away Team Initials"], axis=1)
y = []

In [6]:
for i in range(len(compressed_final)):
    home_team_goals = compressed_final.iloc[i]["Home Team Goals"]
    away_team_goals = compressed_final.iloc[i]["Away Team Goals"]
    
    if home_team_goals > away_team_goals:
        y.append(1)
    elif home_team_goals < away_team_goals:
        y.append(2)
    else:
        y.append(0)

In [7]:
# Test
assert len(X) == len(y)

### Encode textual features from the `X` dataset

In [8]:
X["Stage"] = LabelEncoder().fit_transform(X["Stage"])
X["Home Team Name"] = LabelEncoder().fit_transform(X["Home Team Name"])
X["Away Team Name"] = LabelEncoder().fit_transform(X["Away Team Name"])

In [9]:
len(X.columns)

413

### Feature Selection

In [10]:
X.columns[4:]

Index(['Player 1 Age Diff', 'Player 1 Overall Diff', 'Player 1 Potential Diff',
       'Player 1 Acceleration Diff', 'Player 1 Aggression Diff',
       'Player 1 Agility Diff', 'Player 1 Balance Diff',
       'Player 1 Ball control Diff', 'Player 1 Composure Diff',
       'Player 1 Crossing Diff',
       ...
       'Player 11 Shot power Diff', 'Player 11 Sliding tackle Diff',
       'Player 11 Sprint speed Diff', 'Player 11 Stamina Diff',
       'Player 11 Standing tackle Diff', 'Player 11 Strength Diff',
       'Player 11 Vision Diff', 'Player 11 Volleys Diff',
       'Mean Home Team Goals', 'Mean Away Team Goals'],
      dtype='object', length=409)

In [11]:
# selection = SelectKBest(score_func=f_classif, k=20)
# selection.fit(X, y)

# X = selection.transform(X)

In [12]:
feature_names = [
    "Stage", "Home Team Name", "Away Team Name",
    "Attendance", "Overall",
    "Mean Home Team Goals", "Mean Away Team Goals"
]

COLUMNS = []

for column_name in X.columns:
    for feature_name in feature_names:
        if feature_name in column_name:
            COLUMNS.append(column_name)
            break

X = X[COLUMNS]

### Split `X` and `y` into train / test sets

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Fast testing

In [14]:
def test_model(model, cv=10):
    cv_scores = cross_validate(model, X, y, cv=cv)
    
    mean_train_acc = mean(cv_scores["train_score"]) 
    mean_test_acc = mean(cv_scores["test_score"])
    
    print()
    print("Train Accuracy: ", mean_train_acc)
    print("Test Accuracy: ", mean_test_acc)
    print()
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

### K-Nearest Neighbors

In [15]:
test_model(KNeighborsClassifier(n_neighbors=3))


Train Accuracy:  0.7097997538643426
Test Accuracy:  0.3898809523809524

             precision    recall  f1-score   support

          0       0.19      0.30      0.23        10
          1       0.47      0.41      0.44        22
          2       0.25      0.20      0.22        15

avg / total       0.34      0.32      0.33        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


### Random Forests

In [16]:
model = RandomForestClassifier(n_estimators=1200, max_depth=10, bootstrap=True, n_jobs=-1)

test_model(model)

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)



Train Accuracy:  1.0
Test Accuracy:  0.6955402930402931

             precision    recall  f1-score   support

          0       0.57      0.40      0.47        10
          1       0.80      0.73      0.76        22
          2       0.75      1.00      0.86        15

avg / total       0.74      0.74      0.73        47



### Support Vector Machines

In [17]:
test_model(SVC(C=1.0, kernel="rbf", gamma="auto"))


Train Accuracy:  1.0
Test Accuracy:  0.4087087912087912

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        10
          1       1.00      0.14      0.24        22
          2       0.34      1.00      0.51        15

avg / total       0.58      0.38      0.27        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Extremely Randomized Trees

In [18]:
test_model(ExtraTreesClassifier(n_estimators=1200, max_depth=10, bootstrap=True, n_jobs=-1))

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)



Train Accuracy:  1.0
Test Accuracy:  0.7079212454212453

             precision    recall  f1-score   support

          0       0.62      0.50      0.56        10
          1       0.80      0.73      0.76        22
          2       0.74      0.93      0.82        15

avg / total       0.74      0.74      0.74        47



### Gradient Boosting Machines

In [19]:
model = GradientBoostingClassifier(n_estimators=100, max_depth=3, learning_rate=0.1)

test_model(model)


Train Accuracy:  1.0
Test Accuracy:  0.7027701465201465

             precision    recall  f1-score   support

          0       0.50      0.20      0.29        10
          1       0.65      0.68      0.67        22
          2       0.70      0.93      0.80        15

avg / total       0.64      0.66      0.63        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


### XGBoost (Best for now with 78.53%)

In [20]:
# best_model = XGBClassifier(n_estimators=4000, max_depth=20, learning_rate=0.03, n_jobs=-1)

model = XGBClassifier(n_estimators=4000, max_depth=20, learning_rate=0.03, n_jobs=-1)

test_model(model, cv=10)

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/


Train Accuracy:  1.0
Test Accuracy:  0.6736355311355311

             precision    recall  f1-score   support

          0       0.44      0.40      0.42        10
          1       0.75      0.68      0.71        22
          2       0.78      0.93      0.85        15

avg / total       0.69      0.70      0.69        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### AdaBoost with Decision Tree

In [21]:
tree = DecisionTreeClassifier()
ada = AdaBoostClassifier(tree, n_estimators=10**7, learning_rate=0.03)

test_model(ada)


Train Accuracy:  1.0
Test Accuracy:  0.48383699633699634

             precision    recall  f1-score   support

          0       0.57      0.40      0.47        10
          1       0.74      0.64      0.68        22
          2       0.62      0.87      0.72        15

avg / total       0.66      0.66      0.65        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


### Neural Network

In [22]:
test_model(
    MLPClassifier(
        hidden_layer_sizes=(20, 40, 60, 100, 200, 300, 500, 500, 300, 200, 100, 60, 40, 20),
        activation="logistic"
    )
)

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)



Train Accuracy:  0.4084551476850277
Test Accuracy:  0.4087087912087912

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        10
          1       0.47      1.00      0.64        22
          2       0.00      0.00      0.00        15

avg / total       0.22      0.47      0.30        47



/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Build up a Random Forest Classifier with Grid Search

In [22]:
model = RandomForestClassifier()

grid_search = GridSearchCV(
    model,
    param_grid={
        "n_estimators": [100, 200, 300, 500, 700, 1000],
        "max_depth": [1, 2, 3, 5, 10],
    },
    scoring="accuracy",
    cv=3,
    verbose=True
)

grid_search.fit(X, y)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


KeyboardInterrupt: 

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_params_